In [10]:
import pandas as pd
import numpy as np

# Load the Excel file and examine its structure
file_path = '../data/landing/moving_annual_rent/moving_annual_median_weekly_rent_by_suburb.xlsx'
# Get all sheet names
xl_file = pd.ExcelFile(file_path)
sheet_names = xl_file.sheet_names
print("Available sheets:", sheet_names)


Available sheets: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house', 'All properties']


In [11]:
def extract_median_data(sheet_name):
    """
    Extract median rent data from a specific sheet.
    Returns a DataFrame with suburbs as rows and quarters as columns.
    """
    # Read the raw data
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Find the header row (row 1 contains quarter names)
    quarter_row = 1
    median_row = 2
    
    # Extract quarter names and identify median columns
    quarters = []
    median_cols = []
    
    for col in range(df.shape[1]):
        if pd.notna(df.iloc[quarter_row, col]) and df.iloc[quarter_row, col] != 'Lease commenced in year ending ':
            quarter_name = df.iloc[quarter_row, col]
            if df.iloc[median_row, col] == 'Median':
                quarters.append(quarter_name)
                median_cols.append(col)
    
    # Extract suburb names (starting from row 3)
    suburb_names = []
    data_rows = []
    
    for row in range(3, df.shape[0]):
        if pd.notna(df.iloc[row, 1]):  # Suburb name is in column 1
            suburb_names.append(df.iloc[row, 1])
            # Extract median values for this suburb
            median_values = []
            for col in median_cols:
                value = df.iloc[row, col]
                # Convert '-' to NaN and other values to numeric
                if value == '-' or pd.isna(value):
                    median_values.append(np.nan)
                else:
                    try:
                        median_values.append(float(value))
                    except (ValueError, TypeError):
                        median_values.append(np.nan)
            data_rows.append(median_values)
    
    # Create DataFrame
    result_df = pd.DataFrame(data_rows, index=suburb_names, columns=quarters)
    result_df.index.name = 'Suburb'
    
    return result_df

# Test with the first sheet
test_df = extract_median_data('1 bedroom flat')
print(f"Shape: {test_df.shape}")
print(f"Columns (first 10): {test_df.columns[:10].tolist()}")
print(f"Index (first 10): {test_df.index[:10].tolist()}")
print("\nFirst few rows and columns:")
print(test_df.iloc[:5, :5])


Shape: (159, 101)
Columns (first 10): ['Mar 2000', 'Jun 2000', 'Sep 2000', 'Dec 2000', 'Mar 2001', 'Jun 2001', 'Sep 2001', 'Dec 2001', 'Mar 2002', 'Jun 2002']
Index (first 10): ['Albert Park-Middle Park-West St Kilda', 'Armadale', 'Carlton North', 'Carlton-Parkville', 'CBD-St Kilda Rd', 'Collingwood-Abbotsford', 'Docklands', 'East Melbourne', 'East St Kilda', 'Elwood']

First few rows and columns:
                                       Mar 2000  Jun 2000  Sep 2000  Dec 2000  \
Suburb                                                                          
Albert Park-Middle Park-West St Kilda     165.0     165.0     170.0     175.0   
Armadale                                  150.0     150.0     155.0     160.0   
Carlton North                             150.0     155.0     150.0     150.0   
Carlton-Parkville                         165.0     170.0     175.0     180.0   
CBD-St Kilda Rd                           250.0     250.0     250.0     250.0   

                               

In [12]:
# Process all property type sheets (excluding 'All properties')
property_types = [sheet for sheet in sheet_names if sheet != 'All properties']
print(f"Processing {len(property_types)} property types: {property_types}")

# Combine all property types into one DataFrame
all_dataframes = []

for prop_type in property_types:
    print(f"Processing {prop_type}...")
    df = extract_median_data(prop_type)
    # Add property type column
    df['Property_Type'] = prop_type
    # Reset index to make suburb a regular column
    df = df.reset_index()
    all_dataframes.append(df)

# Combine all DataFrames
combined_df = pd.concat(all_dataframes, ignore_index=True)
print(f"\nCombined DataFrame shape: {combined_df.shape}")
print(f"Columns: {combined_df.columns.tolist()}")
print(f"Property types: {combined_df['Property_Type'].unique()}")
print(f"Number of suburbs per property type:")
print(combined_df['Property_Type'].value_counts())


Processing 6 property types: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house']
Processing 1 bedroom flat...
Processing 2 bedroom flat...
Processing 3 bedroom flat...
Processing 2 bedroom house...
Processing 3 bedroom house...
Processing 4 bedroom house...

Combined DataFrame shape: (954, 103)
Columns: ['Suburb', 'Mar 2000', 'Jun 2000', 'Sep 2000', 'Dec 2000', 'Mar 2001', 'Jun 2001', 'Sep 2001', 'Dec 2001', 'Mar 2002', 'Jun 2002', 'Sep 2002', 'Dec 2002', 'Mar 2003', 'Jun 2003', 'Sep 2003', 'Dec 2003', 'Mar 2004', 'Jun 2004', 'Sep 2004', 'Dec 2004', 'Mar 2005', 'Jun 2005', 'Sep 2005', 'Dec 2005', 'Mar 2006', 'Jun 2006', 'Sep 2006', 'Dec 2006', 'Mar 2007', 'Jun 2007', 'Sep 2007', 'Dec 2007', 'Mar 2008', 'Jun 2008', 'Sep 2008', 'Dec 2008', 'Mar 2009', 'Jun 2009', 'Sep 2009', 'Dec 2009', 'Mar 2010', 'Jun 2010', 'Sep 2010', 'Dec 2010', 'Mar 2011', 'Jun 2011', 'Sep 2011', 'Dec 2011', 'Mar 2012', 'Jun 2012', 'Sep 2012', 'Dec 2012', 

In [14]:
# Display sample of the final DataFrame
print("Sample of the combined DataFrame:")
print(combined_df[['Suburb', 'Property_Type'] + list(combined_df.columns[1:6])].head(10))

print("\nData types:")
print(combined_df.dtypes.head(10))

# Check for missing values
print(f"\nMissing values per column (first 10 quarters):")
quarter_cols = [col for col in combined_df.columns if col not in ['Suburb', 'Property_Type']]
print(combined_df[quarter_cols[:10]].isnull().sum())

# Summary statistics
print(f"\nSummary statistics for recent quarters:")
recent_quarters = [col for col in quarter_cols if '2024' in col or '2025' in col]
if recent_quarters:
    print(combined_df[recent_quarters].describe())
else:
    print("No recent quarters found, showing last available quarters:")
    print(combined_df[quarter_cols[-5:]].describe())


Sample of the combined DataFrame:
                                  Suburb   Property_Type  Mar 2000  Jun 2000  \
0  Albert Park-Middle Park-West St Kilda  1 bedroom flat     165.0     165.0   
1                               Armadale  1 bedroom flat     150.0     150.0   
2                          Carlton North  1 bedroom flat     150.0     155.0   
3                      Carlton-Parkville  1 bedroom flat     165.0     170.0   
4                        CBD-St Kilda Rd  1 bedroom flat     250.0     250.0   
5                 Collingwood-Abbotsford  1 bedroom flat     135.0     140.0   
6                              Docklands  1 bedroom flat       NaN       NaN   
7                         East Melbourne  1 bedroom flat     180.0     180.0   
8                          East St Kilda  1 bedroom flat     140.0     145.0   
9                                 Elwood  1 bedroom flat     150.0     152.0   

   Sep 2000  Dec 2000  Mar 2001  
0     170.0     175.0     180.0  
1     155.0     1

In [ ]:
# Save the processed data
output_path = '../data/processed/moving_annual_rent_long.csv'
combined_df.to_csv(output_path, index=False)
print(f"Data saved to: {output_path}")

# Also create a pivot version for easier analysis
pivot_df = combined_df.set_index(['Suburb', 'Property_Type']).stack().reset_index()
pivot_df.columns = ['Suburb', 'Property_Type', 'Quarter', 'Median_Rent']
pivot_df = pivot_df.sort_values(['Suburb', 'Property_Type', 'Quarter'])

pivot_output_path = '../data/processed/moving_annual_rent_pivot.csv'
pivot_df.to_csv(pivot_output_path, index=False)
print(f"Pivot data saved to: {pivot_output_path}")

print(f"\nFinal DataFrame Summary:")
print(f"- Total records: {len(combined_df)}")
print(f"- Unique suburbs: {combined_df['Suburb'].nunique()}")
print(f"- Property types: {combined_df['Property_Type'].nunique()}")
print(f"- Quarters covered: {len(quarter_cols)}")
print(f"- Date range: {quarter_cols[0]} to {quarter_cols[-1]}")


In [17]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime

# Select 3 interesting suburbs for plotting
selected_suburbs = ['CBD-St Kilda Rd', 'Carlton-Parkville', 'Richmond-Burnley']
print(f"Selected suburbs for time series analysis: {selected_suburbs}")

# Check data availability for selected suburbs
print(f"\nData availability for selected suburbs:")
for suburb in selected_suburbs:
    suburb_data = pivot_df[pivot_df['Suburb'] == suburb]
    if len(suburb_data) > 0:
        print(f"{suburb}: {suburb_data['Property_Type'].nunique()} property types, {suburb_data['Quarter'].nunique()} quarters")
        print(f"  Latest quarter: {suburb_data['Quarter'].max()}")
        print(f"  Rent range: ${suburb_data['Median_Rent'].min():.0f} - ${suburb_data['Median_Rent'].max():.0f}")
    else:
        print(f"{suburb}: No data found")


Selected suburbs for time series analysis: ['CBD-St Kilda Rd', 'Carlton-Parkville', 'Richmond-Burnley']

Data availability for selected suburbs:
CBD-St Kilda Rd: 3 property types, 101 quarters
  Latest quarter: Sep 2024
  Rent range: $250 - $1100
Carlton-Parkville: 6 property types, 101 quarters
  Latest quarter: Sep 2024
  Rent range: $165 - $1200
Richmond-Burnley: 6 property types, 101 quarters
  Latest quarter: Sep 2024
  Rent range: $140 - $1300


In [33]:
# Fix the pivot_df sorting to be chronological
print("Before sorting - first few quarters:")
print(pivot_df['Quarter'].unique()[:10])

# Convert quarters to datetime for proper sorting, then sort chronologically
pivot_df['Quarter_Date'] = pd.to_datetime(pivot_df['Quarter'], format='%b %Y')
pivot_df = pivot_df.sort_values(['Suburb', 'Property_Type', 'Quarter_Date']).reset_index(drop=True)

print("\nAfter sorting - first few quarters:")
print(pivot_df['Quarter'].unique()[:10])

# Save the properly sorted data
pivot_output_path = '../data/processed/moving_annual_rent_pivot.csv'
pivot_df.to_csv(pivot_output_path, index=False)
print(f"\nProperly sorted pivot data saved to: {pivot_output_path}")


Before sorting - first few quarters:
['Dec 2000' 'Dec 2001' 'Dec 2002' 'Dec 2003' 'Dec 2004' 'Dec 2005'
 'Dec 2006' 'Dec 2007' 'Dec 2008' 'Dec 2009']

After sorting - first few quarters:
['Mar 2000' 'Jun 2000' 'Sep 2000' 'Dec 2000' 'Mar 2001' 'Jun 2001'
 'Sep 2001' 'Dec 2001' 'Mar 2002' 'Jun 2002']

Properly sorted pivot data saved to: ../data/processed/moving_annual_rent_pivot.csv


In [22]:
pivot_df.Property_Type.unique()

array(['1 bedroom flat', '2 bedroom flat', '2 bedroom house',
       '3 bedroom flat', '3 bedroom house', '4 bedroom house'],
      dtype=object)

In [27]:
pivot_df.Median_Rent.isna().sum()

np.int64(0)

In [34]:
# Create time series plots for each property type
property_types = pivot_df.Property_Type.unique()

# Create subplots for each property type
fig = make_subplots(
    rows=len(property_types), 
    cols=1, 
    subplot_titles=property_types,
    vertical_spacing=0.1,
    shared_xaxes=True
)

# Add traces for each suburb and property type
for i, prop_type in enumerate(property_types):
    for j, suburb in enumerate(selected_suburbs):
        # Filter data for this suburb and property type
        data = pivot_df[(pivot_df['Suburb'] == suburb) & 
                       (pivot_df['Property_Type'] == prop_type) & 
                       (pivot_df['Median_Rent'].notna())]
        
        if len(data) > 0:
            # Convert quarter strings to datetime and sort by time
            data = data.copy()
            data['Quarter_Date'] = pd.to_datetime(data['Quarter'], format='%b %Y')
            data = data.sort_values('Quarter_Date')  # Ensure proper chronological order
            
            fig.add_trace(
                go.Scatter(
                    x=data['Quarter_Date'],
                    y=data['Median_Rent'],
                    name=f"{suburb}",
                    mode='lines+markers',
                    line=dict(color=colors[j % len(colors)], width=2),
                    marker=dict(size=4),
                    showlegend=(i == 0),  # Only show legend for first subplot
                    connectgaps=False,  # Don't connect across missing data
                    hovertemplate=f"<b>{suburb}</b><br>" +
                                 "Quarter: %{x}<br>" +
                                 "Rent: $%{y:.0f}<br>" +
                                 "<extra></extra>"
                ),
                row=i+1, col=1
            )

# Update layout
fig.update_layout(
    height=3200,
    title_text="Moving Annual Median Weekly Rent Time Series",
    title_x=0.5,
    title_font_size=16,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

# Update x-axis labels
fig.update_xaxes(title_text="Quarter Lease Start", row=len(property_types), col=1)
fig.update_xaxes(showticklabels=True)

# Update y-axis labels for each subplot
for i in range(len(property_types)):
    fig.update_yaxes(title_text="Median weekly rent", row=i+1, col=1)

fig.show()


In [35]:
# Create an alternative view: All property types for one suburb
fig2 = go.Figure()

suburb_to_plot = 'CBD-St Kilda Rd'  # Focus on CBD area
colors_prop = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

all_property_types = pivot_df['Property_Type'].unique()

for i, prop_type in enumerate(all_property_types):
    data = pivot_df[(pivot_df['Suburb'] == suburb_to_plot) & 
                   (pivot_df['Property_Type'] == prop_type) & 
                   (pivot_df['Median_Rent'].notna())]
    
    if len(data) > 0:
        data = data.copy()
        data['Quarter_Date'] = pd.to_datetime(data['Quarter'], format='%b %Y')
        data = data.sort_values('Quarter_Date')  # Ensure proper chronological order
        
        fig2.add_trace(
            go.Scatter(
                x=data['Quarter_Date'],
                y=data['Median_Rent'],
                mode='lines+markers',
                name=prop_type,
                line=dict(color=colors_prop[i % len(colors_prop)], width=2),
                marker=dict(size=5),
                connectgaps=False,  # Don't connect across missing data
                hovertemplate=f"<b>{prop_type}</b><br>" +
                             "Quarter: %{x}<br>" +
                             "Rent: $%{y:.0f}<br>" +
                             "<extra></extra>"
            )
        )

fig2.update_layout(
    title=f"Moving Annual Median Weekly Rent - {suburb_to_plot}",
    title_x=0.5,
    xaxis_title="Quarter Lease Start",
    yaxis_title="Median weekly rent",
    height=600,
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

fig2.show()
